<a href="https://colab.research.google.com/github/Tahimi/AntColonyOptimizationModel/blob/main/AntSimulatorLabirinto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Simulator = {
    'antsNBR':300} #70

In [ ]:
### Problem's Geometry ###
ProblemGEE = {
    'nx':100,
    'ny':70}

In [ ]:
### read wall Lower and Higher Bounds, LB and HB ###
ProblemGEE.update({
    # cost from analytical function
    'costFromAnalFunc':False,
    # cost from input text file
    'fread_2Dcost':False,
    '2Dcost_file':'2Dcost_file.csv',
    # cost from image
    'costFromImage':True,
    'cell_data':"Declividade.jpeg", #"labirinto1_rotated.jpeg", #"Declividade.jpeg",
    # lower and higher bounds of cost for wall
    'costLB':0.,
    'costHB':255.,
    'homogenize_cost':False})

In [ ]:
ProblemLabirintito = {
    'nx':100,
    'ny':100}

In [ ]:
ProblemLabirintito.update({
    # cost from analytical function
    'costFromAnalFunc':False,
    # cost from input text file
    'fread_2Dcost':False,
    '2Dcost_file':'2Dcost_file.csv',
    # cost from image
    'costFromImage':True,
    'cell_data':"labirinto1_rotated.jpeg",
    # lower and higher bounds of cost for wall
    'costLB':4.,
    'costHB':255.,
    'homogenize_cost':True})

In [ ]:
Problem = ProblemLabirintito

In [ ]:
import os, sys
print(os.getcwd())
sys.path.insert(0, os.getcwd())

C:\Users\PC\Desktop\UFAL\Pesquisa\AntSimulatorPython\AntSimulator_2D\Jupyter


In [ ]:
import numpy as np
from PIL import Image, ImageOps

def readImage(image):
    originalImage = Image.open(image)
    #originalImage.show()
    reducedImage = originalImage.resize((Problem['ny'],Problem['nx']))    
    grayImage = ImageOps.grayscale(reducedImage)
    elevation = np.array(grayImage)
    return elevation

In [ ]:
import numpy as np
import pandas as pd
import math

In [ ]:
### Nest and Food cells ###
Problem.update({
    'nest':[1,Problem['ny']//2],
    'food':[Problem['nx']-2,Problem['ny']//2] })

In [ ]:
### Cost np.array: random values ###
Problem.update({
    'cost':np.random.default_rng().integers(low=Problem['costLB'],
                                            high=Problem['costHB'],
                                            size=(Problem['nx'],Problem['ny']),
                                            endpoint=True).astype(float) })

In [ ]:
Problem['cost']=readImage(Problem['cell_data'])
#print("Problem['cost']=",Problem['cost'])

In [ ]:
print("Problem['cost'].shape = ", Problem['cost'].shape)

Problem['cost'].shape =  (100, 100)


In [ ]:
### evaluating a given analytical function for cost at cell ###
def funcAtCell(cellIdx):
    ubCoords = unitBoxCoords(cellIdx)
    x = ubCoords[0]; y = ubCoords[1]
    return 2.*(20. * math.exp(-((x-0.5)**2+(y-0.5)**2))-17.)

In [ ]:
### cell center coordenates ###
def unitBoxCoords(cellIdx):
    nx = ProblemData['nx']; ny = ProblemData['ny']
    return np.array(cellIdx) / np.array([nx,ny]) + np.array([1./(2.*nx), 1./(2.*ny)])

In [ ]:
### filling the cost nparray from the analytical function ###
def costFromAnalFunc():
    nx = Problem['nx']; ny = Problem['ny']
    for i in range(nx):
        for j in range(ny):
            Problem['cost'][i][j] = funcAtCell([i,j])
if(Problem['costFromAnalFunc']): costFromAnalFunc()

In [ ]:
### filling the cost nparray from a given input file ###
def fread_2Dcost():
    Problem['cost'][:,:]=pd.read_csv(Problem['2Dcost_file'],sep="\s+",header=None).to_numpy().astype(float)
if(Problem['fread_2Dcost']): fread_2Dcost()

In [ ]:
def walls():
    cost=Problem['cost']; lb=Problem['costLB']; hb=Problem['costHB']
    return np.where(np.logical_and((cost>lb),(cost<hb)),False,True) #todo: invert

In [ ]:
Problem.update({
    'wall':walls()
})

In [ ]:
if(Problem['homogenize_cost']): Problem['cost'][:,:]=1.

In [ ]:
### convert nparrays to python lists ###
#Problem['cost'] = Problem['cost'].tolist()
#Problem['wall'] = Problem['wall'].tolist()
#print('\nProblem\n----------\n',Problem)

In [ ]:
Simulator.update({
    'alpha':1.,
    'betha':1.,
    'gamma':1.,
    'rho':0.005,
    'phi':0.05,
    'neighborsNBR':9,
    'cyclesNBR':70,
    'cyclesPeriod':50,
    'dt':1.,
    'tmax':50.,
    'initialFeromone':np.array([1,1],dtype=float),
    'INFTY':1000000.,
    'EPSILON':1./1000000.})
#print('\Simulator\n----------\n',Simulator)

In [ ]:
from enum import Enum, unique

In [ ]:
@unique
class AntRole(Enum):
    followToFoodFeromone = 1 #pg.Color('green4')
    followToNestCostMax = 2 #pg.Color('black')
    fleeToNestFeromone = 3 #pg.Color('red')
    followToNestCostMin = 4 #pg.Color('darkorange4')

In [ ]:
@unique
class stats(Enum):
    cost = 1
    toNestCost = 2
    toNestFeromone = 3
    toFoodFeromone = 4
    all = 5

In [ ]:
import numpy as np
import random
import math
import timeit
import datetime
import matplotlib.pyplot as plt

In [ ]:
class Cell:
    maxCost = np.sum(Problem['cost'])
    nestToFoodMinStepsNBR = np.amax(np.abs(np.array(Problem['food'])-np.array(Problem['nest'])))

    def __init__(self,Idx):
        self.Idx = np.array(Idx)
        self.isFree = True 
        self.cost = Problem['cost'][Idx[0]][Idx[1]]
        self.toNestCost = 0. if self.isNest() else Cell.maxCost
        self.feromone = np.copy(Simulator['initialFeromone'])
        self.fenceBoundary()
    
    def fenceBoundary(self):
        i = self.Idx[0]; j = self.Idx[1]
        if( (i*j==0) or (i==nx-1) or (j==ny-1) ):
            Problem['wall'][i][j] = True
        
    def isFood(self):
        return np.array_equal(np.array(Problem['food']),self.Idx)
        
    def isNest(self):
        return np.array_equal(np.array(Problem['nest']),self.Idx)

    def onWall(self):
        i = self.Idx[0]; j = self.Idx[1]
        return Problem['wall'][i][j]

    def coords(self):
        return np.array(self.Idx)+1/2.

    def normalizedCoords(self):
        return self.coords()/ np.array([nx,ny])

    def __eq__(self, other):
        return np.array_equal(other.Idx,self.Idx)

In [ ]:
nx = Problem['nx']; ny = Problem['ny']

In [ ]:
class gridCells:
    def __init__(self):
        self.grid = np.ndarray(shape=(nx,ny),dtype=Cell)
        for i in range(self.grid.shape[0]):
            for j in range(self.grid.shape[1]):
                self.grid[i,j] = Cell([i,j])

    def cell(self,Idx):
        i=Idx[0];j=Idx[1]
        return self.grid[i][j]
    
    def update_wall(self,cell):
        if cell.isNest() or cell.isFood(): return
        idx = cell.Idx; i = idx[0]; j = idx[1]
        Problem['wall'][i][j] = not Problem['wall'][i][j]

In [ ]:
gcs = gridCells()

In [ ]:
class Segment:
    def __init__(self,cellA,cellB,idx):
        self.Idx = idx
        self.A = cellA
        self.B = cellB
        self.isDummy = np.array_equal(self.A.Idx,self.B.Idx)
        self.cost = (self.A.cost+self.B.cost)/2.

    def isUnviable(self):
        return (self.A.onWall() or self.B.onWall() or self.isDummy)
        
    def __eq__(self, other):
        if(not other): return False
        return (self.A==other.A and self.B==other.B)

    def isInverseOf(self,other):
        if(not other): return False
        return (self.A==other.B and self.B==other.A)

In [ ]:
class gridSegments:
    def __init__(self,gcs):
        self.grid = np.ndarray(shape=(nx,ny),dtype=object)
        self.gcs = gcs
        ## todo create a feromone ndarray for all segments

    def append(self,AIdx,BIdx,segIdx):
        self.grid[AIdx[0]][AIdx[1]] = np.append(self.grid[AIdx[0]][AIdx[1]],
            [ Segment(self.gcs.cell(AIdx),self.gcs.cell(BIdx),segIdx) ])

    def feromone(self,segment):
        #if(segment.isUnviable()): return np.array([0,0])
        return segment.B.feromone

    def AtoBIdx(self,segment):
        return self.appendingIdx(segment.B.Idx - segment.A.Idx)

    def BtoAIdx(self,segment):
        return self.appendingIdx(segment.A.Idx - segment.B.Idx)

    # todo keep deleting the 3d dimension tomorrow. 19/11/21
    def appendingIdx(self,idx):
        ii=idx[0];jj=idx[1]
        return (ii+1)*3+(jj+1)

    def inverseOf(self,segment):
        BIdx = segment.B.Idx
        return self.grid[BIdx[0]][BIdx[1]][self.BtoAIdx(segment)]

In [ ]:
gss = gridSegments(gcs)
for i in range(1,nx-1):
    for j in range(1,ny-1):
        for ii in [-1,0,1]:
            for jj in [-1,0,1]:
                segIdx = i*3+j+(ii+1)*3+(jj+1)
                gss.append([i,j],[i+ii,j+jj],segIdx)
        gss.grid[i][j] = np.delete(gss.grid[i][j],0)

In [ ]:
class Ant:
    alpha = Simulator['alpha']
    betha = Simulator['betha']
    gamma = Simulator['gamma']
    INFTY = Simulator['INFTY']
    EPSILON = Simulator['EPSILON']
    neighborsNBR = Simulator['neighborsNBR']

    def __init__(self,Idx,nestCell):
        self.Idx = Idx
        self.route = []
        self.cyclesNBR = 0
        self.cycleSize = 0
        self.bestPath = []
        self.minToNestCost = Cell.maxCost
        self.proposeColonyBestPath = False
        self.stoppingCounter = 0 # sometimes it get stuck because of recentelyvisitedcells
        self.isSelected = False
        self.isRetired = False
        self.status = {'role':AntRole.followToNestCostMax,
                       'toFood':True,
                       'currCell':nestCell,'nextCell':None,'nextSegment':None}

    def step(self):
        if self.status['currCell'].isFood() or self.status['currCell'].isNest():
            self.initializeNewCycle()
        self.identifyNextCell()
        if(self.status['nextCell']) is not None:
            self.proceed()

    def identifyNextCell(self):
        # initialize with no nextCell found
        self.status['nextCell'] = None
        # no need to identify for retired ants
        if self.isRetired: return
        # the creteria depends on wether the ant is going to food or to nest.
        if self.status['toFood']: self.identifyNextCellToFood()
        else: self.identifyNextCellToNest()

    def proceed(self):
        if self.cycleSize==0: self.updateNeighborhood()
        self.cycleSize += 1
        self.moveToNextCell()
        self.updateNeighborhood()
        self.updateRoute()
        if self.reachedGoal():
            self.cyclesNBR +=1
            self.updateBestPath()
            self.status['toFood'] = not self.status['toFood']
            if not self.isRetired: self.isRetired = self.isSelected and not self.status['currCell'].isFood()

    def recentelyVisited(self,cellB,n = 4): # Cell.nestToFoodMinStepsNBR for the ant to avoid to be stuck in a short loop.
        if self.stoppingCounter > 1: return False
        for seg in self.route[len(self.route)-n:][::-1]:
            if(cellB == seg.A or cellB == seg.B): return True
        return False

    def dFeromone(self):
        return (np.array([0,1]) if self.status['toFood'] else np.array([1,0]))

    def moveToNextCell(self):
        # free the cell and go to the next one
        self.status['currCell'].isFree = True
        self.status['currCell'] = self.status['nextCell']
        # occupy the new cell except if it is food or nest, then let it be free for other ants to access it.
        if (self.status['currCell'].isNest() or self.status['currCell'].isFood()):
            self.status['currCell'].isFree = True
        else: self.status['currCell'].isFree = False

    def updateNeighborhood(self):
        AIdx = self.status['currCell'].Idx
        fromAToNestCost = self.status['currCell'].toNestCost
        self.status['currCell'].feromone += self.dFeromone()
        data = np.zeros(shape=(self.neighborsNBR,),dtype=float)
        for segment in gss.grid[AIdx[0]][AIdx[1]]:
            segment.B.toNestCost = min(segment.B.toNestCost, fromAToNestCost + segment.B.cost)
            segment.B.feromone += self.dFeromone()

    def updateRoute(self):
        self.route = np.append(self.route,self.status['nextSegment'])

    def updateBestPath(self):
        if not self.status['currCell'].isFood():
            lastCycleCost = self.routeCost(self.route)
            if(lastCycleCost <= self.minToNestCost):
                self.minToNestCost = lastCycleCost
                self.bestPath = np.copy(self.route)
                self.proposeColonyBestPath = True
                self.printBestPath()

    def initializeNewCycle(self):
        self.route = []
        self.cycleSize = 0

    def reachedGoal(self):
        return (self.reachedFood() or self.reachedNest())

    def reachedFood(self):
        return (self.status['currCell'].isFood() and self.status['toFood'])

    def reachedNest(self):
        return (self.status['currCell'].isNest() and not self.status['toFood'])

    def routeCost(self,route):
        sumCost = 0.
        for segment in route: sumCost += segment.cost
        return sumCost

    def printBestPath(self):
        if(self.cyclesNBR==0): return
        print(r"ant's minToNestCost = ",self.minToNestCost,end=', ')
        self.printRoute(self.bestPath,name='antBestPath: ')

    def printRoute(self,route,name='givenRoute'):
        if(len(route)==0): return
        print(r'ant.Idx = ',self.Idx,', cyclesNBR = ',self.cyclesNBR,', len(route): ',len(route), end='')
        print(r', ', name, end='')
        for segment in route:
            print(r'(',segment.A.Idx,',',segment.B.Idx,')', end='; ')        
        print()

In [ ]:
class Ant(Ant):
    def identifyNextCellToFood(self):
        self.fleeToNestFeromone()
        
    def fleeToNestFeromone(self):
        self.status['role'] = AntRole.fleeToNestFeromone
        AIdx = self.status['currCell'].Idx
        data = np.zeros(shape=(self.neighborsNBR,),dtype=float)
        for i,segment in enumerate(gss.grid[AIdx[0]][AIdx[1]]):
            tho = gss.feromone(segment)[1]
            etha = 1./max(gss.feromone(segment)[0], 1.) #segment.cost
            if np.random.rand() < 0.8: data[i] = gss.feromone(segment)[1]
            else: data[i] = tho**self.alpha * etha**self.betha
            #data[i] = gss.feromone(segment)[1]
            if segment.isUnviable(): data[i] = Ant.INFTY # these data shouldn't interfere
        denominator = np.sum(data)
        if denominator > Ant.EPSILON: data /= denominator        
        orderedDataIdx = np.lexsort((np.random.random(len(data)), data))[::1] # [::1] means from min to max
        for i in orderedDataIdx:
            nextSegmentCandidate = gss.grid[AIdx[0]][AIdx[1]][i]
            if nextSegmentCandidate.isUnviable(): continue
            if self.recentelyVisited(nextSegmentCandidate.B): continue
            nextCellCandidate = gss.grid[AIdx[0]][AIdx[1]][i].B
            if(nextCellCandidate.isFree):
                self.status['nextCell'] = nextCellCandidate
                self.status['nextSegment'] = gss.grid[AIdx[0]][AIdx[1]][i]
                self.stoppingCounter = 0
                return True
        self.stoppingCounter += 1
        return False

In [ ]:
class Ant(Ant):
    def identifyNextCellToNest(self):
        self.followToNestCostMin()
        
    def followToNestCostMin(self):
        self.status['role'] = AntRole.followToNestCostMin
        AIdx = self.status['currCell'].Idx
        data = np.zeros(shape=(self.neighborsNBR,),dtype=float)
        for i,segment in enumerate(gss.grid[AIdx[0]][AIdx[1]]):
            data[i] = segment.B.toNestCost
            if segment.isUnviable(): data[i] = Ant.INFTY # these data shouldn't interfere
        denominator = np.sum(data)
        if denominator > Ant.EPSILON: data /= denominator
        orderedDataIdx = np.lexsort((np.random.random(len(data)), data))[::1] # [::1] means from min to max
        for i in orderedDataIdx:
            nextSegmentCandidate = gss.grid[AIdx[0]][AIdx[1]][i]
            if nextSegmentCandidate.isUnviable(): continue
            nextCellCandidate = gss.grid[AIdx[0]][AIdx[1]][i].B
            if(nextCellCandidate.isFree or not self.status['toFood']):
                self.status['nextCell'] = nextCellCandidate
                self.status['nextSegment'] = gss.grid[AIdx[0]][AIdx[1]][i]
                return True
        return False

In [ ]:
class AntsColony:
    size = Simulator['antsNBR']
    def __init__(self,gcs):
        self.gcs = gcs
        self.bestPath = []
        self.secondBestPath = []
        self.feromoneEvaporationRate = 0.
        self.minToNestCost = Cell.maxCost
        self.ants = np.ndarray((AntsColony.size,),dtype=Ant)
        for i in range(AntsColony.size):
            self.ants[i] = Ant(i,self.gcs.cell(Problem['nest']))
        
    def update_nestCell(self,cell):        
        #print("Problem['nest'] = ",Problem['nest'])
        A = self.gcs.cell(Problem['nest'])
        self.moveFromAtoB(A,cell)
        idx = cell.Idx; i = idx[0]; j = idx[1]
        Problem['nest'] = cell.Idx
        Problem['wall'][i][j] = False
        #print("Problem['nest'] = ",Problem['nest'])
        
    def moveFromAtoB(self,A,B):
        A.toNestCost = Cell.maxCost
        B.toNestCost = 0.
        for i in range(AntsColony.size):
            self.ants[i].status['currCell'] = B
        
    def update_foodCell(self,cell):
        print("Problem['food'] = ",Problem['food'])
        idx = cell.Idx; i = idx[0]; j = idx[1]
        Problem['food'] = idx
        Problem['wall'][i][j] = False
        print("Problem['food'] = ",Problem['food'])
        
    def updateBestPath(self,ant):
        if(ant.minToNestCost < self.minToNestCost):
            self.minToNestCost = ant.minToNestCost
            self.secondBestPath = np.copy(self.bestPath)
            self.bestPath = np.copy(ant.bestPath)
            self.printBestPath()
            return True
        return False

    def printBestPath(self):
        print(r"colony's minToNestCost = ",self.minToNestCost,end=', ')
        for segment in self.bestPath:
            print(r'(',segment.A.Idx,',',segment.B.Idx,')', end='; ')
        print()

    def evaporateFeromone(self):
        if len(self.bestPath) == 0: return
        self.feromoneEvaporationRate = 0.01/len(self.bestPath)
        grid = self.gcs.grid
        for i in range(grid.shape[0]):
            for j in range(grid.shape[1]):
                    grid[i,j].feromone *= max((1. - self.feromoneEvaporationRate),0.) # k=1 for dim==2

In [ ]:
import pygame as pg
from pygame.locals import *

pygame 2.1.0 (SDL 2.0.16, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
nx = Problem['nx']
ny = Problem['ny']
#nest = Problem['nest']
#food = Problem['food']

In [ ]:
class Colors:
    cellPerimeter = pg.Color('black')
    text = pg.Color('black')
    wall = pg.Color('grey40')
    domain = pg.Color('grey99')
    food = pg.Color('green');
    nest = pg.Color('orange')

    # ant's color
    followToFoodFeromone = pg.Color('green4')
    followToNestCostMax = pg.Color('black')
    fleeToNestFeromone = pg.Color('red')
    followToNestCostMin = pg.Color('darkorange4')
    secondBestPath = pg.Color('yellow')
    bestPath = pg.Color('blue')

    # route's color
    toFood = nest
    toNest = food

    # colorBar
    minValue = pg.Color('white') # pg.Color('gray99')
    #maxValue = pg.Color('black')
    maxValue = pg.Color('black') # pg.Color('darkslategray')

In [ ]:
class DisplayerManager:
    #def __init__(self,gcs,gss,antsColony,width=960,height=720,name=f'Ant Simulator'):
    #def __init__(self,gcs,gss,antsColony,width=(4/3)*960,height=(4/3)*720,name=f'Ant Simulator'):
    def __init__(self,gcs,gss,antsColony,width=3*620,height=3*330,name=f'Ant Simulator'):
    #def __init__(self,gcs,gss,antsColony,width=1820,height=980,name=f'Ant Simulator'):
        self.gcs = gcs
        self.gss = gss
        self.antsColony = antsColony
        self.width = width
        self.height = height
        self.name = name
        self.cell_width = width//nx
        self.cell_height = height//ny

        pg.init()
        pg.font.init()
        self.window = pg.display.set_mode((self.width,self.height))
        pg.display.set_caption(self.name)
        self.clock = pg.time.Clock()
        self.initialDisplay = True
        self.display = True

        self.antImg = pg.image.load('ant.png').convert_alpha()
        antHeight = self.cell_height
        antWidth = int(antHeight * self.antImg.get_width()/self.antImg.get_height())
        self.antImg = pg.transform.scale(self.antImg, (antWidth, antHeight))

        self.xOffset = (self.cell_width - self.antImg.get_width())//2
        self.yOffset = (self.cell_height - self.antImg.get_height())//2

        self.minExposedDataValue = Simulator['INFTY']
        self.maxExposedDataValue = 0.
        self.spanOfData = self.maxExposedDataValue - self.minExposedDataValue
        self.selectedPath = []

    def run(self):
        i = 0
        while self.initialDisplay:
            print("press the 'p' botton to start.")
            self.update()
        
    def routeColor(self,ant):
        return Colors.toFood if ant.status['toFood'] else Colors.toNest

    def antColor(self,ant):
        if ant.status['role'] == AntRole.followToFoodFeromone: return Colors.followToFoodFeromone
        elif ant.status['role'] == AntRole.followToNestCostMax: return Colors.followToNestCostMax
        elif ant.status['role'] == AntRole.fleeToNestFeromone: return Colors.fleeToNestFeromone
        elif ant.status['role'] == AntRole.followToNestCostMin: return Colors.followToNestCostMin

    def update(self):
        self.draw()
        self.processEvent()
        self.clock.tick(3)

    def quit(self):
        pg.quit()

    def draw_retiredAntsLastPaths(self,color):
        for ant in self.antsColony.ants:
            if not ant.isRetired: continue
            self.draw_route(ant.route,color)

    def draw_domain(self):
        self.window.fill(Colors.domain)

    def updateDataSpan(self,data):
        self.minExposedDataValue = Simulator['INFTY']
        self.maxExposedDataValue = -Simulator['INFTY']
        for i in range(0,nx):
            for j in range(0,ny):
                cell = self.gcs.grid[i][j]
                if cell.onWall(): continue
                if data == stats.toNestCost:
                    if cell.toNestCost < self.minExposedDataValue: self.minExposedDataValue = cell.toNestCost
                    if cell.toNestCost > self.maxExposedDataValue: self.maxExposedDataValue = cell.toNestCost
                if data == stats.cost:
                    if cell.cost < self.minExposedDataValue: self.minExposedDataValue = cell.cost
                    if cell.cost > self.maxExposedDataValue: self.maxExposedDataValue = cell.cost
                if data == stats.toFoodFeromone:
                    if cell.toFoodFeromone < self.minExposedDataValue: self.minExposedDataValue = cell.toFoodFeromone
                    if cell.toFoodFeromone > self.maxExposedDataValue: self.maxExposedDataValue = cell.toFoodFeromone
                if data == stats.toNestFeromone:
                    if cell.toNestFeromone < self.minExposedDataValue: self.minExposedDataValue = cell.toNestFeromone
                    if cell.toNestFeromone > self.maxExposedDataValue: self.maxExposedDataValue = cell.toNestFeromone
        self.spanOfData = self.maxExposedDataValue - self.minExposedDataValue

    def draw_cellData(self,data = None):
        if not data: return
    
        self.updateDataSpan(data)        
        for i in range(0,nx):
            for j in range(0,ny):
                cell = self.gcs.grid[i][j]
                if cell.onWall(): continue
                if data == stats.toNestCost:
                    dataValue = cell.toNestCost
                elif data == stats.cost:
                    dataValue = cell.cost
                elif data == stats.toFoodFeromone:
                    dataValue = cell.feromone[0]
                elif data == stats.toNestFeromone:
                    dataValue = cell.feromone[1]
                else: return
                
                coeff = (dataValue - self.minExposedDataValue)/self.spanOfData
                color = Colors.minValue.lerp(Colors.maxValue, coeff)
                if Problem['homogenize_cost']: color = pg.Color('white')
                xCell = i * self.cell_width
                yCell = j * self.cell_height
                self.draw_rectangle(color,xCell,yCell)

    def plot_cellDataContour(self,data = None):
        if not data: return
        self.updateDataSpan(data)
        z = np.zeros(shape=(nx,ny),dtype=float)
        #x = np.zeros(shape=(nx,),dtype=float)
        #y = np.zeros(shape=(ny,),dtype=float)
        for i in range(0,nx):
            for j in range(0,ny):
                cell = self.gcs.grid[i][j]
                if cell.onWall(): continue
                if data == stats.toNestCost:
                    dataValue = cell.toNestCost
                elif data == stats.cost:
                    dataValue = cell.cost
                #elif data == stats.toFoodFeromone:
                #    dataValue = cell.feromone[0]
                #elif data == stats.toNestFeromone:
                #    dataValue = cell.feromone[1]
                else: return
                z[i][j] = dataValue

        #fig = go.Figure(data = go.Contour(
        #    z = z,
        #    x = np.arange(nx, dtype = float), # horizontal axis
        #    y = np.arange(ny, dtype = float), # vertical axis
        #    colorscale='Hot',
        #    contours_coloring='lines',
        #    line_width=2,
        #    contours=dict(
        #        coloring ='heatmap',
        #        showlabels = True, # show labels on contours
        #        labelfont = dict( # label font properties
        #            size = 12,
        #            color = 'black' ),
        #        start=self.minExposedDataValue,
        #        end=self.maxExposedDataValue,
        #        size=(self.maxExposedDataValue-self.minExposedDataValue)/10.,
        #        )))
        fig = go.Figure(data =
            go.Contour(
                z= z,
                dx=1,
                x0=0,
                dy=1,
                y0=0,
            ))
        #fig.update_layout(legend_traceorder="reversed")
        name = ""
        if data == stats.toNestCost: name = "toNestCost"
        elif data == stats.cost: name = "cost"
        fig.write_html(name+".html", auto_open=True)

    def draw_cell(self,color,x,y,board_thikness=1):
        pg.draw.rect(self.window, color, pg.Rect(x,y,self.cell_width,self.cell_height),board_thikness)

    def text_cell(self,data,x,y,board_thikness=1):
        font=pg.font.SysFont('timesnewroman', 12)
        text = font.render(data, True, Colors.text, Colors.domain)
        textRect = text.get_rect()
        textRect.center = (x + self.cell_width//2, y + self.cell_height//2)
        self.window.blit(text, textRect)

    def cellAtScreenPos(self,pos):
        print(r'pos[0] = ',pos[0], ', pos[1] = ',pos[1])
        i=pos[0]//self.cell_width; j=pos[1]//self.cell_height
        print(r'i = ',i, ', j = ',j)
        print(r'cellIdx = ',self.gcs.grid[i][j].Idx)
        return self.gcs.grid[i][j]

    def antAtCell(self,cell):
        for ant in self.antsColony.ants:
            if ant.status['currCell'] == cell:
                return ant
            else: return None

    def draw_nest(self):
        nest = Problem['nest']
        xNest = nest[0] * self.cell_width
        yNest = nest[1] * self.cell_height
        self.draw_rectangle(Colors.nest,xNest,yNest)

    def draw_food(self):
        food = Problem['food']
        xfood = food[0] * self.cell_width
        yfood = food[1] * self.cell_height
        self.draw_rectangle(Colors.food,xfood,yfood)

    def draw_ants(self):
        for ant in self.antsColony.ants:
            if ant.isRetired: continue
            self.draw_ant(ant)

    def draw_ant(self,ant):
        #self.draw_route(ant.route,color=self.routeColor(ant))
        self.colorAnt(self.antColor(ant))
        self.blitRotatedAnt(ant)

    def draw_route(self,route,color):
        if(len(route)==0): return
        for segment in route:
            cell = segment.B
            if cell.isNest() or cell.isFood(): continue
            xCell = cell.Idx[0] * self.cell_width
            yCell = cell.Idx[1] * self.cell_height
            self.draw_rectangle(color,xCell,yCell)

    def blitRotatedAnt(self,ant):
        BIdx = ant.status['currCell'].Idx

        xCell = BIdx[0] * self.cell_width
        yCell = BIdx[1] * self.cell_height
        #self.draw_cell(wall_color,xCell,yCell)

        image = self.antImg
        xant = xCell + self.xOffset
        yant = yCell + self.yOffset
        ##self.window.blit(image, (xant, yant))

        antRect = image.get_rect()
        antRect.move_ip(xant,yant)
        #pg.draw.rect(self.window, ant.color(), antRect, 1)

        image = pg.transform.rotozoom(image, self.orientation(ant), 1)
        self.window.blit(image, image.get_rect(center = antRect.center))

    def orientation(self,ant):
        if not len(ant.route): return 0.
        AIdx = ant.route[-1].A.Idx
        BIdx = ant.route[-1].B.Idx
        dx = BIdx[0] - AIdx[0]
        dy = BIdx[1] - AIdx[1]
        return math.degrees(-math.atan2(dy, dx)-math.pi/2.)

    def colorAnt(self, color):
        """Fill all pixels of the antImg with color, preserve transparency."""
        w, h = self.antImg.get_size()
        r, g, b, _ = color
        for x in range(w):
            for y in range(h):
                a = self.antImg.get_at((x, y))[3]
                self.antImg.set_at((x, y), pg.Color(r, g, b, a))

    def draw_rectangle(self,color,x,y):
        pg.draw.rect(self.window, color, pg.Rect(x,y,self.cell_width,self.cell_height))

In [ ]:
class DisplayerManager(DisplayerManager):
    def draw_walls(self):
        for i in range(self.gcs.grid.shape[0]):
            for j in range(self.gcs.grid.shape[1]):
                cell = self.gcs.grid[i,j]
                if cell.onWall(): self.draw_wall(cell)

    def draw_wall(self,cell):
        idx = cell.Idx; x = idx[0]; y = idx[1] 
        xCell = x * self.cell_width
        yCell = y * self.cell_height
        self.draw_rectangle(Colors.wall,xCell,yCell)

In [ ]:
class DisplayerManager(DisplayerManager):
    def processEvent(self):
        for event in pg.event.get():
            if event.type == pg.QUIT:
                self.display = False
            elif event.type == pg.KEYDOWN:
                if event.key == pg.K_SPACE: ## space tab to freez the simulation/execution
                    resumeComupation = False
                    while not resumeComupation:
                        for evnt2 in pg.event.get():
                            if evnt2.type == pg.KEYDOWN:
                                if evnt2.key == pg.K_r: ## press r to resume / unfreez
                                    resumeComupation = True
                                if evnt2.key == pg.K_u:
                                    self.draw_domain()
                                    self.draw_walls()
                                    self.draw_cellData(stats.cost)
                                    self.print_cellData(stats.all)
                                    pg.display.flip()
                                if evnt2.key == pg.K_c:
                                    self.plot_cellDataContour(stats.cost)
                                if evnt2.key == pg.K_n:
                                    self.plot_cellDataContour(stats.toNestCost)

                            elif evnt2.type == pg.MOUSEBUTTONDOWN:
                                cell = self.cellAtScreenPos(pg.mouse.get_pos())
                                ant = self.antAtCell(cell)
                                if ant is None:
                                    print("cell is selected, cell.Idx = ",cell.Idx)
                                    
                                    if evnt2.button == 1:
                                        print("You pressed the left mouse button: Wall")
                                        self.gcs.update_wall(cell)
                                    elif evnt2.button == 2:
                                        print("You pressed the mouse wheel button: Food")
                                        self.antsColony.update_foodCell(cell)
                                    elif evnt2.button == 3:
                                        print("You pressed the right mouse button: Nest")
                                        self.antsColony.update_nestCell(cell)
                            
                                    self.draw()
                                else:
                                    if not cell.isNest():
                                        print("ant is selected, ant.Idx = ",ant.Idx)
                                        print("np.array(Problem['nest']) = ", np.array(Problem['nest']))
                                        print("cell.Idx = ", cell.Idx)
                                        print("cell.isNest() = ", cell.isNest())
                                        ant.isSelected = True
                # stop initial display in order to run the timeManager
                elif event.key == pg.K_p:
                    self.initialDisplay = False
                    print('### key p pressed.')

In [ ]:
class DisplayerManager(DisplayerManager):
    def draw(self):
        self.draw_domain()
        self.draw_walls()
        self.draw_cellData(stats.cost)
        self.draw_nest()
        self.draw_food()
        self.draw_route(self.antsColony.secondBestPath,Colors.secondBestPath)
        self.draw_route(self.antsColony.bestPath,Colors.bestPath)
        self.draw_retiredAntsLastPaths(Colors.bestPath)
        self.draw_ants()
        #self.print_cellData(stats.all)
        pg.display.flip()

In [ ]:
class DisplayerManager(DisplayerManager):
    def print_cellData(self,data = None):
        if not data: return
        for i in range(0,nx):
            for j in range(0,ny):
                cell = self.gcs.grid[i][j]
                if cell.onWall(): continue
                xCell = i * self.cell_width
                yCell = j * self.cell_height
                self.draw_cell(Colors.cellPerimeter,xCell,yCell)
                if data == stats.toNestCost:
                    dataValue = str(int(cell.toNestCost))
                elif data == stats.cost:
                    dataValue = str(int(cell.cost))
                elif data == stats.toFoodFeromone:
                    dataValue = str(int(cell.feromone[0]))
                elif data == stats.toNestFeromone:
                    dataValue = str(int(cell.feromone[1]))
                elif data == stats.all:
                    #dataValue = "("+str(int(cell.cost))+";"+str(int(cell.toNestCost))+";"+str(int(cell.feromone[0]))+";"+str(int(cell.feromone[1]))+")"
                    dataValue = "("+str(int(cell.cost))+";"+str(int(cell.toNestCost))+")"
                else: return
                self.text_cell(dataValue,xCell,yCell)

In [ ]:
class TimeManager:
    def __init__(self):
        self.time = 0.
        self.it = 0
        self.dt=Simulator['dt']
        self.tmax=Simulator['tmax']
        self.simStartInstant = timeit.default_timer()
        self.bestPathTimeSpan_Cost = []
        self.finalTimeSpan = 0
        self.antsColony = AntsColony(gcs)
        self.displayerManager = DisplayerManager(gcs,gss,self.antsColony)

    def run(self):
        # diplay the domain to setup the Problem, adjust the walls, define nest and food cells.
        self.displayerManager.run()
        #self.displayerManager.quit()

        #while self.time < 5: #self.tmax:
        while self.displayerManager.display:
            print("time = ",self.time)
            self.step()
            self.displayerManager.update()
            #value = input("Please enter a string to continue:\n")
        self.displayerManager.quit()

        #for ant in antsColony.ants:
        #    ant.printRoute(ant.route,name='Full route: ')

        for ant in self.antsColony.ants:
            #ant.printBestPath()
            print(r'ant.Idx = ',ant.Idx,', cyclesNBR = ',ant.cyclesNBR, ", minToNestCost = ",ant.minToNestCost)
            
        self.antsColony.printBestPath()
        print(r'self.bestPathTimeSpan_Cost = ',self.bestPathTimeSpan_Cost)
        self.plotBestPathTimeSpan_Cost()
        print(r'Last best path reached within: ',self.finalTimeSpan)
        #print(r'self.displayerManager.minExposedDataValue = ', self.displayerManager.minExposedDataValue)
        #print(r'self.displayerManager.maxExposedDataValue = ', self.displayerManager.maxExposedDataValue)

    def step(self):
        for ant in self.antsColony.ants:
            print("ant.Idx = ",ant.Idx)
            ant.step()
            if ant.proposeColonyBestPath:
                isUpdated = self.antsColony.updateBestPath(ant)
                if isUpdated:
                    deltaT = timeit.default_timer() - self.simStartInstant
                    self.finalTimeSpan = datetime.timedelta(seconds = deltaT)
                    self.bestPathTimeSpan_Cost.append([deltaT,self.antsColony.minToNestCost])
                    ant.proposeColonyBestPath = False
        self.antsColony.evaporateFeromone()
        self.time += self.dt

    def plotBestPathTimeSpan_Cost(self):
        x, y = zip(*self.bestPathTimeSpan_Cost)
        plt.plot(x, y, '.r-')
        plt.xlabel('Time (sec)')
        plt.ylabel('Price (R$)')
        titleTxt = 'Mesh:'+str(Problem['nx'])+' x '+str(Problem['ny'])+' ; Ants number:'+str(Simulator['antsNBR'])
        plt.title(titleTxt)
        plt.show()

In [ ]:
tManager = TimeManager()

In [ ]:
tManager.run()